In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

df = pd.read_csv(r"D:\Yess_Info\Project\helpdesk\dataset\ticket_dataset.csv")
df.head()

,subject,body,answer,type,queue,priority,language,version,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,Outage,Disruption,Data Breach,NaN,NaN,NaN,NaN
1,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,Disruption,Outage,IT,Tech Support,NaN,NaN,NaN
2,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,Feature,Tech Support,NaN,NaN,NaN,NaN,NaN
3,Inquiry Regarding Invoice Details,"Dear Customer Support Team,\n\nI hope this mes...",We appreciate you reaching out with your billi...,Request,Billing and Payments,low,en,51,Billing,Payment,Account,Documentation,Feedback,NaN,NaN,NaN
4,Question About Marketing Agency Software Compa...,"Dear Support Team,\n\nI hope this message reac...",Thank you for your inquiry. Our product suppor...,Problem,Sales and Pre-Sales,medium,en,51,Product,Feature,Feedback,Tech Support,NaN,NaN,NaN,NaN


In [3]:
text_cols = ['subject','body','answer']

for col in text_cols:
    df[col] = df[col].astype(str).str.replace('\n',' ', regex=False).str.replace('\r',' ', regex=False).str.strip()


In [4]:
tag_cols = [c for c in df.columns if c.startswith("tag_")]

def merge_tags(row):
    tags = []
    for col in tag_cols:
        val = str(row[col]).strip()
        if val not in ["", "nan", "None"]:
            tags.append(val)
    return tags

df["tags"] = df.apply(merge_tags, axis=1)
df["tag_count"] = df["tags"].apply(len)


In [5]:
df.insert(0, "ticket_id", ["TKT-"+str(100000+i) for i in range(len(df))])


In [6]:
np.random.seed(42)

start_date = datetime.now() - timedelta(days=365)
created_dates = []

for i in range(len(df)):
    random_days = np.random.randint(0, 365)
    random_hours = np.random.randint(0, 24)
    dt = start_date + timedelta(days=random_days, hours=random_hours)
    created_dates.append(dt)

df["created_date"] = created_dates


In [7]:
sla_map = {"low":48, "medium":24, "high":8}

df["priority"] = df["priority"].str.lower()
df["sla_hours"] = df["priority"].map(sla_map)


In [8]:
resolution = []

for sla in df["sla_hours"]:
    if pd.isna(sla):
        sla = 24
    
    mean = sla * np.random.uniform(0.6, 1.5)  # some within SLA, some not
    val = np.random.normal(mean, sla*0.5)
    val = max(1, min(val, 7*24))  # max 7 days
    resolution.append(round(val,2))

df["resolution_hours"] = resolution

df["closed_date"] = df["created_date"] + pd.to_timedelta(df["resolution_hours"], unit="h")


In [9]:
df["sla_breached"] = np.where(df["resolution_hours"] > df["sla_hours"], "Yes", "No")


In [10]:
df["body_word_count"] = df["body"].apply(lambda x: len(str(x).split()))
df["subject_char_count"] = df["subject"].str.len()


In [11]:
df.head(3)

,ticket_id,subject,body,answer,type,queue,priority,language,version,tag_1,...,tag_8,tags,tag_count,created_date,sla_hours,resolution_hours,closed_date,sla_breached,body_word_count,subject_char_count
0,TKT-100000,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,...,NaN,"[Security, Outage, Disruption, Data Breach]",4,2025-02-26 14:16:43.223062,8,11.13,2025-02-27 01:24:31.223062,Yes,82,31
1,TKT-100001,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,...,NaN,"[Account, Disruption, Outage, IT, Tech Support]",5,2025-10-30 09:16:43.223062,8,10.30,2025-10-30 19:34:43.223062,Yes,82,18
2,TKT-100002,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,...,NaN,"[Product, Feature, Tech Support]",3,2025-03-02 02:16:43.223062,24,31.35,2025-03-03 09:37:43.223062,Yes,76,50


In [12]:
df.isna().sum()

ticket_id                 0
subject                   0
body                      0
answer                    0
type                      0
queue                     0
priority                  0
language                  0
version                   0
tag_1                     0
tag_2                    13
tag_3                   136
tag_4                  3058
tag_5                 14042
tag_6                 22713
tag_7                 26547
tag_8                 28022
tags                      0
tag_count                 0
created_date              0
sla_hours                 0
resolution_hours          0
closed_date               0
sla_breached              0
body_word_count           0
subject_char_count        0
dtype: int64

In [13]:
df["created_date"] = pd.to_datetime(df["created_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
df["closed_date"] = pd.to_datetime(df["closed_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")

In [14]:
tag_cols = ["tag_1","tag_2","tag_3","tag_4","tag_5","tag_6","tag_7","tag_8"]

for col in tag_cols:
    df[col] = df[col].fillna("None")


In [15]:
df.head(3)

,ticket_id,subject,body,answer,type,queue,priority,language,version,tag_1,...,tag_8,tags,tag_count,created_date,sla_hours,resolution_hours,closed_date,sla_breached,body_word_count,subject_char_count
0,TKT-100000,Wesentlicher Sicherheitsvorfall,"Sehr geehrtes Support-Team,\n\nich möchte eine...",Vielen Dank für die Meldung des kritischen Sic...,Incident,Technical Support,high,de,51,Security,...,None,"[Security, Outage, Disruption, Data Breach]",4,2025-02-26 14:16:43,8,11.13,2025-02-27 01:24:31,Yes,82,31
1,TKT-100001,Account Disruption,"Dear Customer Support Team,\n\nI am writing to...","Thank you for reaching out, <name>. We are awa...",Incident,Technical Support,high,en,51,Account,...,None,"[Account, Disruption, Outage, IT, Tech Support]",5,2025-10-30 09:16:43,8,10.30,2025-10-30 19:34:43,Yes,82,18
2,TKT-100002,Query About Smart Home System Integration Feat...,"Dear Customer Support Team,\n\nI hope this mes...",Thank you for your inquiry. Our products suppo...,Request,Returns and Exchanges,medium,en,51,Product,...,None,"[Product, Feature, Tech Support]",3,2025-03-02 02:16:43,24,31.35,2025-03-03 09:37:43,Yes,76,50


In [19]:
df.to_csv("D:\Yess_Info\Project\helpdesk\python(pandas)\it_helpdesk_cleaned.csv", index=False)